In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
import keras
from sklearn.preprocessing import LabelEncoder
import re
from sklearn import preprocessing
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax

In [ ]:
data = pd.read_pickle('../input/all_tainan_data.pkl')
data.reset_index(drop=True, inplace=True) 

In [ ]:
data.info()

In [ ]:
x_train = data[['base_area', 'building_area', 'height', 'latitude'\
               , 'longitude', 'number_basements', 'number_design_changes', 'number_floors'\
              ,'number_households', 'total_floor_area', 'liquefaction','e_PGA', 'e_PGV', 'e_PGD', 'n_PGA', 'n_PGV'\
              ,'n_PGD', 'u_PGA', 'u_PGV', 'u_PGD']]
y_train = data['damage_level']

In [ ]:
dates = []
for item in data.issuing_license_number:
    date = re.findall('\([0-9]+\)|$', item)[0]
    date = re.sub('\(|\)', '', date)
    if date == '':
        date = '61'
    dates += [int(date)]

In [ ]:
x_train['date'] = dates

In [ ]:
def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

In [ ]:
category_binarizer = preprocessing.LabelBinarizer()
category_binarizer.fit(data.category)
category_binarizer.transform(data['category'])
binary_category = category_binarizer.transform(data['category'])

In [ ]:
#todo represent this differently
construction_type_binarizer = preprocessing.LabelBinarizer()
construction_type_binarizer.fit(data.construction_type)
binary_construction = construction_type_binarizer.transform(data['construction_type'])

In [ ]:
license_category_binarizer = preprocessing.LabelBinarizer()
license_category_binarizer.fit(data.license_category)
binary_license = license_category_binarizer.transform(data['license_category'])

In [ ]:
x_train.info()

In [ ]:
y_train_encoder = LabelEncoder()
y_train_encoder.fit(y_train)
y_train = label_encode(y_train_encoder, y_train)

In [ ]:
#prepare x_train
x_train = x_train.values

In [ ]:
print("Shape x_train:",x_train.shape)
print("Shape binary_category:",binary_category.shape)
print("Shape binary_construction:",binary_construction.shape)
print("Shape binary_license:",binary_license.shape)

In [ ]:
#concatenate all categorical attributes after binarization
x_train = np.concatenate((x_train,binary_category), axis = 1)
x_train = np.concatenate((x_train,binary_construction), axis = 1)
x_train = np.concatenate((x_train,binary_license), axis = 1)

In [ ]:
x_train.shape

In [ ]:
# I/O check
input_shape = x_train.shape[1]
print('input_shape: ', input_shape)

output_shape = len(y_train_encoder.classes_)
print('output_shape: ', output_shape)

In [ ]:

# input layer
model_input = Input(shape=(input_shape, ))  # 500
X = model_input

# 1st hidden layer
X_W1 = Dense(units=128)(X)  # 64
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=128)(H1)  # 64
H2 = ReLU()(H1_W2)

# output layer
H2_W3 = Dense(units=output_shape)(H2)  # 4
H3 = Softmax()(H2_W3)

model_output = H3

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

In [ ]:
epochs = 5
batch_size = 32

# training!
history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)
print('training finish')

In [ ]:
#predict
pred_result = model.predict(x_test, batch_size = 128)
pred_result = label_decode(y_train_encoder, pred_result)

In [ ]:
collapse = 0
danger = 0
no_danger = 0
for item in pred_result:
    if item == 1:
        danger += 1
    if item == 2:
        collapse += 1
    if item == 0:
        no_danger += 1
print(danger)
print(collapse)
print(no_danger)
print(len(pred_result))